# Data Cleansing


The purpose of this notebook is to take the information from each image and insert it into a Postgres database as a single row.

In [37]:
MIKE_ORIGINALS = r'C:\DS Project\Images\Mike\Original'
#MIKE_ORIGINALS = r'C:\DS Project\Images\Mike\test'
NEIL_ORIGINALS = r'C:\DS Project\Images\Neil\Original'
MANUALLY_IDENTIFIED_TRACTORS =  r'C:\DS Project\Images\Tractors'

SET_SECONDS = 2

## Imports

In [38]:
import psycopg2
import db_config as creds
import os
import time
import datetime
from stat import * 
from PIL import Image

In [39]:
class theImg:
  def __init__(self, filename, createdDate,cameraName,cameraId,folderLocation, origWidth, origHeight):
    self.createdDate = createdDate
    self.filename = filename
    self.cameraName = cameraName
    self.cameraId = cameraId
    self.folderLocation = folderLocation
    self.origWidth = origWidth
    self.origHeight = origHeight
        
    # see if the image has previously been identifed as a tractor
    self.MWIdentified = False
    for subdir, dirs, files in os.walk(MANUALLY_IDENTIFIED_TRACTORS):
        for tractorFilename in files:
            if tractorFilename == filename:
                self.MWIdentified = True
    

In [40]:
def processImages(imgLocation, cameraName, cameraId):
    imgSet = 1
    imagesInSet = []

    # loop round each image in the original folder
    for subdir, dirs, files in os.walk(imgLocation):
        
        for filename in files:
        
            filepath = subdir + os.sep + filename
            
            stat = os.stat(filepath)
            createdDate = datetime.datetime.fromtimestamp(stat[ST_MTIME])
            
            # get dimensions
            im = Image.open(filepath)
            w, h = im.size
        
            # create new image object with attributes
            newImg = theImg(filename,createdDate,cameraName,cameraId,subdir + os.sep,w,h)
        
            # check if image is > SET_SECONDS seconds offset the last image
            if (len(imagesInSet) > 0) and ( (createdDate-imagesInSet[len(imagesInSet)-1].createdDate).total_seconds() > SET_SECONDS ):
            
                # insert each of the images in the list into the database
                addSetToDB(imagesInSet)
                
                # increment the set counter
                imgSet+=1
            
                # clear out the list
                imagesInSet.clear()
        
            #set the image set 
            newImg.imgSet = imgSet    
            
            # add the image to the list
            imagesInSet.append(newImg)
        
    # add the last set to the data base     
    addSetToDB(imagesInSet) 

## Database functions

In [41]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)

    # Create a cursor object
    cursor = conn.cursor()
  
    #print ("connected!")
    return conn, cursor

def disconnect(conn,cursor):
    
    cursor.close()
    conn.close()
    #print ("disconnected!")
    
def addSetToDB(imgSet):
    
    sql = """INSERT INTO images ("filename", "cameraid", "cameraname", "datetaken", "setid", "mw_manuallyidentifiedtractor", "folderlocation", "orig_width", "orig_height") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        
    # Connecting to DB
    conn, cursor = connect()
    
    for img in imgSet:

        #insert into db
        cursor.execute(sql, (img.filename,img.cameraId,img.cameraName, img.createdDate, img.imgSet, img.MWIdentified, img.folderLocation, img.origWidth, img.origHeight,))
        conn.commit()
        
        print ("Inserted..",img.filename,img.cameraId,img.cameraName, img.createdDate, img.imgSet, img.MWIdentified, img.folderLocation)
        
    # Disconnect from DB
    disconnect(conn, cursor)

# Process Original Images from Mike and Neil

In [42]:
processImages(MIKE_ORIGINALS, "Mike", 1) 
#processImages(NEIL_ORIGINALS, "Neil", 2) 
print ("done")

Inserted.. 01Mk0005.JPG 1 Mike 2020-03-11 07:10:52 1 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0006.JPG 1 Mike 2020-03-11 07:10:54 1 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0007.JPG 1 Mike 2020-03-11 07:10:56 1 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0008.JPG 1 Mike 2020-03-11 07:10:58 1 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0009.JPG 1 Mike 2020-03-11 07:11:02 2 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0010.JPG 1 Mike 2020-03-11 07:11:02 2 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0011.JPG 1 Mike 2020-03-11 07:11:02 2 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0012.JPG 1 Mike 2020-03-11 07:11:02 2 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0013.JPG 1 Mike 2020-03-11 07:11:

Inserted.. 01Mk0074.JPG 1 Mike 2020-03-11 07:26:46 10 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0075.JPG 1 Mike 2020-03-11 07:27:32 11 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0076.JPG 1 Mike 2020-03-11 07:27:32 11 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0077.JPG 1 Mike 2020-03-11 07:27:32 11 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0078.JPG 1 Mike 2020-03-11 07:34:22 12 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0079.JPG 1 Mike 2020-03-11 07:34:22 12 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0080.JPG 1 Mike 2020-03-11 07:39:38 13 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0081.JPG 1 Mike 2020-03-11 07:39:38 13 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0082.JPG 1 Mike 2020-03-1

Inserted.. 01Mk0144.JPG 1 Mike 2020-03-11 09:08:52 35 True C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0145.JPG 1 Mike 2020-03-11 09:08:52 35 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0146.JPG 1 Mike 2020-03-11 09:08:52 35 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0147.JPG 1 Mike 2020-03-11 09:08:52 35 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0148.JPG 1 Mike 2020-03-11 09:08:54 35 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0149.JPG 1 Mike 2020-03-11 09:08:54 35 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0150.JPG 1 Mike 2020-03-11 09:08:54 35 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0151.JPG 1 Mike 2020-03-11 09:08:54 35 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0152.JPG 1 Mike 2020-03-11

Inserted.. 01Mk0214.JPG 1 Mike 2020-03-11 11:14:42 58 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0215.JPG 1 Mike 2020-03-11 11:15:46 59 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0216.JPG 1 Mike 2020-03-11 11:17:26 60 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0217.JPG 1 Mike 2020-03-11 11:17:26 60 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0218.JPG 1 Mike 2020-03-11 11:17:28 60 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0219.JPG 1 Mike 2020-03-11 11:22:50 61 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0220.JPG 1 Mike 2020-03-11 11:22:50 61 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0221.JPG 1 Mike 2020-03-11 11:22:50 61 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0222.JPG 1 Mike 2020-03-1

Inserted.. 01Mk0286.JPG 1 Mike 2020-03-11 13:12:56 86 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0287.JPG 1 Mike 2020-03-11 13:12:56 86 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0288.JPG 1 Mike 2020-03-11 13:12:56 86 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0289.JPG 1 Mike 2020-03-11 13:12:58 86 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0290.JPG 1 Mike 2020-03-11 13:15:42 87 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0291.JPG 1 Mike 2020-03-11 13:15:42 87 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0292.JPG 1 Mike 2020-03-11 13:15:42 87 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0293.JPG 1 Mike 2020-03-11 13:15:44 87 False C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\
Inserted.. 01Mk0294.JPG 1 Mike 2020-03-1

...

Inserted.. 04Mk4239.JPG 1 Mike 2020-05-10 15:21:32 5219 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4240.JPG 1 Mike 2020-05-10 15:21:32 5219 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4241.JPG 1 Mike 2020-05-10 15:30:14 5220 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4242.JPG 1 Mike 2020-05-10 15:36:14 5221 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4243.JPG 1 Mike 2020-05-10 17:31:32 5222 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4244.JPG 1 Mike 2020-05-10 17:31:32 5222 True C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4245.JPG 1 Mike 2020-05-10 17:31:32 5222 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4246.JPG 1 Mike 2020-05-10 19:01:10 5223 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4247.JPG 1 Mike 2020-05-10 19:01:10 5223 False C:\DS Project\Image

Inserted.. 04Mk4311.JPG 1 Mike 2020-05-11 00:49:22 5265 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4312.JPG 1 Mike 2020-05-11 00:49:26 5266 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4313.JPG 1 Mike 2020-05-11 00:53:48 5267 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4314.JPG 1 Mike 2020-05-11 01:10:58 5268 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4315.JPG 1 Mike 2020-05-11 01:13:36 5269 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4316.JPG 1 Mike 2020-05-11 01:15:24 5270 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4317.JPG 1 Mike 2020-05-11 01:18:34 5271 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4318.JPG 1 Mike 2020-05-11 01:24:34 5272 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4319.JPG 1 Mike 2020-05-11 01:25:14 5273 False C:\DS Project\Imag

Inserted.. 04Mk4384.JPG 1 Mike 2020-05-11 03:38:56 5315 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4385.JPG 1 Mike 2020-05-11 03:38:56 5315 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4386.JPG 1 Mike 2020-05-11 03:45:50 5316 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4387.JPG 1 Mike 2020-05-11 03:48:24 5317 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4388.JPG 1 Mike 2020-05-11 03:50:52 5318 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4389.JPG 1 Mike 2020-05-11 03:50:52 5318 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4390.JPG 1 Mike 2020-05-11 03:55:38 5319 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4391.JPG 1 Mike 2020-05-11 03:55:38 5319 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4392.JPG 1 Mike 2020-05-11 03:56:20 5320 False C:\DS Project\Imag

Inserted.. 04Mk4456.JPG 1 Mike 2020-05-11 04:38:16 5363 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4457.JPG 1 Mike 2020-05-11 04:38:18 5363 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4458.JPG 1 Mike 2020-05-11 04:38:20 5363 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4459.JPG 1 Mike 2020-05-11 04:40:00 5364 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4460.JPG 1 Mike 2020-05-11 04:40:02 5364 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4461.JPG 1 Mike 2020-05-11 04:40:06 5365 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4462.JPG 1 Mike 2020-05-11 04:40:12 5366 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4463.JPG 1 Mike 2020-05-11 04:42:44 5367 False C:\DS Project\Images\Mike\Original\04 May 4 11 2020\
Inserted.. 04Mk4464.JPG 1 Mike 2020-05-11 04:42:46 5367 False C:\DS Project\Imag

OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
